In [14]:
import os
import random
import numpy as np

def game_simulation_with_probabilities(rounds=10000, total_cards=False):
    folder_path = 'card_data'
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print("Folder 'card_data' created.")
    
    def create_hands():
        sequences = []
        for i in range(8):
            seq = [(i >> j) & 1 for j in range(2, -1, -1)]
            sequences.append(seq)
        return sequences

    def binary_to_string(seq):
        return ''.join(['R' if bit == 1 else 'B' for bit in seq])

    def game_simulation(sequences, deck, total_cards=False):
        win_matrix = np.zeros((8, 8))
        for i, player1_seq in enumerate(sequences):
            for j, player2_seq in enumerate(sequences):
                points_player_1, points_player_2 = 0, 0
                cards_player_1, cards_player_2 = 0, 0
                card_pile = []
                k = 0
                while k <= len(deck) - 3:
                    current_sequence = deck[k:k+3]
                    card_pile += deck[k:k+3]

                    if current_sequence == player1_seq:
                        if total_cards:
                            cards_player_1 += len(card_pile)
                            card_pile = []
                        else:
                            points_player_1 += 1
                        k += 3
                    elif current_sequence == player2_seq:
                        if total_cards:
                            cards_player_2 += len(card_pile)
                            card_pile = []
                        else:
                            points_player_2 += 1
                        k += 3
                    else:
                        k += 1

                if total_cards:
                    if cards_player_1 > cards_player_2:
                        win_matrix[i, j] += 1
                    elif cards_player_2 > cards_player_1:
                        win_matrix[i, j] += 0
                else:
                    if points_player_1 > points_player_2:
                        win_matrix[i, j] += 1
                    elif points_player_2 > points_player_1:
                        win_matrix[i, j] += 0

        return win_matrix

    sequences = create_hands()
    overall_win_matrix = np.zeros((8, 8))
    deck_history = []

    for i in range(rounds):
        deck = [1] * 26 + [0] * 26
        random.shuffle(deck)
        deck_history.append(deck)
        win_matrix = game_simulation(sequences, deck, total_cards)
        overall_win_matrix += win_matrix

    win_probabilities = overall_win_matrix / rounds

    with open(os.path.join(folder_path, 'win_probabilities.txt'), 'w') as f:
        f.write("Winning probabilities for each sequence combination (Player 1 vs Player 2):\n")
        for i in range(len(sequences)):
            for j in range(len(sequences)):
                p1_seq = binary_to_string(sequences[i])
                p2_seq = binary_to_string(sequences[j])
                probability = win_probabilities[i, j]
                f.write(f"P1: {p1_seq} vs P2: {p2_seq} --> Probability that P1 wins: {probability:.5f}\n")

    deck_history_file_path = os.path.join(folder_path, 'deck_history.txt')
    if os.path.exists(deck_history_file_path):
        with open(deck_history_file_path, 'r') as f:
            start_index = sum(1 for line in f if line.startswith("Round"))
    else:
        start_index = 0

    with open(os.path.join(folder_path, 'deck_history.txt'), 'a') as f:
        f.write("Deck history for each round (binary 0s and 1s for Black and Red):\n")
        for round_num, deck in enumerate(deck_history, start=start_index+1):
            deck_str = ''.join(['R' if card == 1 else 'B' for card in deck])
            f.write(f"Round {round_num}: {deck_str}\n")

game_simulation_with_probabilities(rounds=3000, total_cards=False)